### Mount Google Drive, Import Libraries and Define Paths

In [1]:
# =============================================================================
# ENVIRONMENT SETUP + PATH CONFIGURATION (SERVER / COLAB COMPATIBLE)
# =============================================================================

import os
import sys
import importlib
from pathlib import Path
import string
import re
import gc
from datetime import timedelta
from scipy.stats.mstats import winsorize

# -----------------------------------------------------------------------------
# 0) HARD SAFETY: cap native thread usage (prevents pthread_create EAGAIN)
#    MUST be set before importing numpy / scipy / pandas
# -----------------------------------------------------------------------------
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_MAX_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["BLIS_NUM_THREADS"] = "1"

# -----------------------------------------------------------------------------
# 1) Detect environment
# -----------------------------------------------------------------------------
IN_COLAB = "google.colab" in sys.modules

# -----------------------------------------------------------------------------
# 2) (Colab only) Mount Google Drive
# -----------------------------------------------------------------------------
if IN_COLAB:
    from google.colab import drive
    drive.mount("/content/drive")
    BASE_PATH = "/content/drive/MyDrive/Colab Notebooks"
else:
    # Server base path (your target)
    BASE_PATH = "/home/jovyan/work/hpool1/pseidel/test"

print("IN_COLAB:", IN_COLAB)
print("BASE_PATH:", BASE_PATH)

# -----------------------------------------------------------------------------
# 3) Sanity checks: path exists + write permission
# -----------------------------------------------------------------------------
BASE = Path(BASE_PATH)
if not BASE.exists():
    raise FileNotFoundError(f"BASE_PATH does not exist: {BASE}")

# quick write test (fails fast if you don't have permissions)
test_file = BASE / ".write_test_tmp"
try:
    test_file.write_text("ok", encoding="utf-8")
    test_file.unlink()
except Exception as e:
    raise PermissionError(f"No write permission in {BASE}. Error: {e}")

# -----------------------------------------------------------------------------
# 4) Environment check: ensure required packages import cleanly
# -----------------------------------------------------------------------------
required_packages = ["numpy", "scipy", "pandas", "linearmodels", "xlsxwriter"]

for pkg in required_packages:
    print(f"Importing {pkg} ...")
    importlib.import_module(pkg)
    print(f"{pkg} OK")

import numpy as np
import pandas as pd

# -----------------------------------------------------------------------------
# 5) Base paths and input/output locations
# -----------------------------------------------------------------------------
Input_file_path   = str(BASE / "Input")
Temp_file_path    = str(BASE / "Temp")
Output_file_path  = str(BASE / "Output")

Fundamentals_file_path = f"{Input_file_path}/WSFV_f_20250131.txt"
Current_file_path      = f"{Input_file_path}/WSCurrent_f_20250131.txt"
Calendar_file_path     = f"{Input_file_path}/WSCalendarPrd_f_20250131.txt"
Meta_file_path         = f"{Input_file_path}/WSMetaData_f_20250131.txt"
Excel_file_path        = f"{Input_file_path}/WS PIT Table Definitions V5 with start dates.xls"

MarketValues_file_path          = f"{Input_file_path}/Daily MV USD"
MarketValues_file_path_LC       = f"{Input_file_path}/Daily MV LC"
DailyTotalReturns_file_path     = f"{Input_file_path}/Daily Returns USD"
DailyIndexReturns_file_path     = f"{Input_file_path}/Daily Index Returns USD"
Constituents_file_path          = f"{Input_file_path}/Constituents.01.csv"
UniversalMatching_file_path     = f"{Input_file_path}/Universal Matching File"

Temp_file_path_GO  = f"{Temp_file_path}/TempGeneralOverview"
Temp_file_path_EoC = f"{Temp_file_path}/TempExtractionofCharacteristics"
Temp_file_path_DP  = f"{Temp_file_path}/TempDataPreparation"
Temp_file_path_A   = f"{Temp_file_path}/TempAnomalies"
Temp_file_path_R   = f"{Temp_file_path}/TempRegressionModel"

Relevant_items_path   = f"{Input_file_path}/RelevantItems.txt"
Relevant_items_path_A = f"{Input_file_path}/RelevantItems.txt"
Relevant_items_path_B = f"{Input_file_path}/RelevantItemsB.txt"
Relevant_items_path_C = f"{Input_file_path}/RelevantItemsC.txt"
Relevant_items_path_D = f"{Input_file_path}/RelevantItemsD.txt"

Subset_file_path = f"{Temp_file_path_GO}/Subsets"
Fundamentals_clean_file_path = f"{Temp_file_path_GO}/Fundamentals_clean.txt"
Current_clean_file_path      = f"{Temp_file_path_GO}/Current_clean.txt"
Calendar_clean_file_path     = f"{Temp_file_path_GO}/Input/Calendar_clean.txt"
Meta_clean_file_path         = f"{Temp_file_path_GO}/Input/Meta_clean.txt"

# -----------------------------------------------------------------------------
# 6) Ensure required directories exist
# -----------------------------------------------------------------------------
Path(Output_file_path).mkdir(parents=True, exist_ok=True)
Path(Temp_file_path_GO).mkdir(parents=True, exist_ok=True)
Path(Temp_file_path_EoC).mkdir(parents=True, exist_ok=True)
Path(Temp_file_path_DP).mkdir(parents=True, exist_ok=True)
Path(Temp_file_path_A).mkdir(parents=True, exist_ok=True)
Path(Temp_file_path_R).mkdir(parents=True, exist_ok=True)
Path(Subset_file_path).mkdir(parents=True, exist_ok=True)
Path(Path(Calendar_clean_file_path).parent).mkdir(parents=True, exist_ok=True)

# -----------------------------------------------------------------------------
# 7) Streaming / deduplication settings
# -----------------------------------------------------------------------------
CHUNK_SIZE = 2_000_000
DATE_COL = "PIT Date"
DEDUP_KEYS = ["ID", "ItemCode", DATE_COL]

print("Paths configured. Temp outputs ->", Temp_file_path_GO)
print("Example input path ->", Fundamentals_file_path)


IN_COLAB: False
BASE_PATH: /home/jovyan/work/hpool1/pseidel/test
Importing numpy ...
numpy OK
Importing scipy ...
scipy OK
Importing pandas ...
pandas OK
Importing linearmodels ...
linearmodels OK
Importing xlsxwriter ...
xlsxwriter OK
Paths configured. Temp outputs -> /home/jovyan/work/hpool1/pseidel/test/Temp/TempGeneralOverview
Example input path -> /home/jovyan/work/hpool1/pseidel/test/Input/WSFV_f_20250131.txt


In [2]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:           754Gi       188Gi       398Gi        55Mi       175Gi       565Gi
Swap:             0B          0B          0B


# Prepare Data

### Merge Fundamentals & Market Data + Roll Forward (Max 2 Calendar Years) + Annual/Daily Deciles or Total/Country/Replication

=> Comment: Previously separate cells but I/O was so intense that I combined it to only load the files once and also switched to .parquet files

In [3]:
# =============================================================================
# MASTER PRODUCTION CELL: UNIFIED ANOMALY PROCESSING PIPELINE
# =============================================================================
#
# SYSTEM ARCHITECTURE:
# --------------------
# This script is a "One-Shot" pipeline. Instead of saving intermediate files to disk
# (which caused the I/O bottleneck), it holds the data for a single anomaly in RAM
# from start to finish.
#
# HARDWARE CONFIGURATION:
# - Workers: 10 (Parallel Processes)
#   * Each worker needs ~30-40GB RAM at peak.
#   * Total Peak RAM: ~400GB (Safe on your 754GB server).
# - Output Format: Apache Parquet (Binary, Compressed, Fast).
#
# DETAILED LOGIC FLOW (Per Anomaly):
# ----------------------------------
# 1. SETUP:
#    - The Main Process loads the "Benchmark" (Returns/Price) and "Factors" (Fama-French)
#      into shared memory so workers don't have to reload them 50 times.
#
# 2. MERGE (The Data Enrichment):
#    - Worker loads ONE anomaly file (e.g., "ACC_clean.txt").
#    - It filters the Benchmark to keep only IDs present in that anomaly file.
#    - It merges them: Market Data + Accounting Signal.
#
# 3. FILL-FORWARD (The Signal Propagation):
#    - Accounting data comes once a year; Returns come daily.
#    - The script "fills forward" the accounting signal from the announcement date
#      to subsequent trading days.
#    - SAFETY RULES:
#      a. Stop filling if the signal is > 2 Years old (730 days).
#      b. Stop filling if there is a trading gap > 14 days (stock suspended/delisted).
#
# 4. RANKING (The Decile Calculation):
#    - DAILY Mode: Ranks are calculated every single day where 'EventDate == 1' (New News).
#    - ANNUAL Mode: Ranks are calculated once per year (June 30th) and held constant.
#    - SCOPES: Ranks are calculated 3 ways: Global, Country-Neutral, and Replication Subset.
#
# 5. FILTER & SAVE (The Output Generation):
#    - The script drops all rows EXCEPT Decile 1 (Top) and Decile 10 (Bottom).
#    - It merges the relevant Fama-French Factors (Global/Country specific).
#    - It saves 6 separate Parquet files per anomaly.
#
# FINAL OUTPUTS:
# --------------
# Total Files = (Num Anomalies) * 6.
# Examples:
# - acc_Global_Daily_clean.parquet
# - acc_Country_Annual_clean.parquet
# =============================================================================

import os
import sys
import gc
import pandas as pd
import numpy as np
from pathlib import Path
from joblib import Parallel, delayed
import time

# --- SAFETY FIX: PREVENT THREAD CRASHES ---
if "NUMEXPR_MAX_THREADS" in os.environ:
    del os.environ["NUMEXPR_MAX_THREADS"]

# =============================================================================
# 1. CONFIGURATION
# =============================================================================

# PATHS
BENCHMARK_FILE = Path(Temp_file_path_DP) / "FF_Benchmark_data_clean.txt"
INPUT_DIR      = Path(Temp_file_path_A) # Folder containing ACC_clean.txt, etc.
FACTOR_DIR     = Path(Temp_file_path_A) # Folder containing Factor files
OUTPUT_DIR     = Path(Temp_file_path_R) # Folder for final .parquet output

# SETTINGS
INPUT_PATTERN  = "*_clean.txt"      # Matches ACC_clean.txt, AG_clean.txt...
SEP            = "|"                # Separator for input text files
TWO_YEARS_DAYS = 730                # Limit for carrying data forward
N_JOBS         = 6                 # Optimized for 754GB RAM
REPLICATION_COUNTRIES = [124, 840]  # Country Codes for Replication subset

# VALIDATION SETTINGS (Sanity Check at the end)
VAL_ID         = "C12486370"
VAL_START      = "2000-06-20"
VAL_END        = "2000-07-10"
VAL_ANOMALY    = "acc"

# COLUMNS TO FORCE AS TEXT (Preserves "0124" vs "124")
STRING_COLS = ["ID", "FiscalPeriod", "HistCurrency", "PCUR", "Country"]

# DATE LIMITS
START_DATE     = "1994-01-01"
END_DATE       = "2024-12-31"

# =============================================================================
# 2. HELPER FUNCTIONS
# =============================================================================

def load_benchmark():
    """Loads the massive Benchmark file. Forces 'Country' to string. Filters Dates."""
    print(f"Loading Benchmark: {BENCHMARK_FILE.name}...")
    dtype_map = {"ID": str, "Country": str} 
    try:
        df = pd.read_csv(BENCHMARK_FILE, sep=SEP, dtype=dtype_map, low_memory=False)
    except:
        df = pd.read_csv(BENCHMARK_FILE, sep=SEP, low_memory=False)
        df["ID"] = df["ID"].astype(str)
        if "Country" in df.columns: df["Country"] = df["Country"].astype(str)

    df["DayDate"] = pd.to_datetime(df["DayDate"], errors="coerce")
    
    # --- NEW FILTERING STEP HERE ---
    print(f"Filtering Benchmark for range: {START_DATE} to {END_DATE}")
    df = df[(df["DayDate"] >= START_DATE) & (df["DayDate"] <= END_DATE)]
    # -------------------------------
    
    if "rf" in df.columns: df.drop(columns=["rf"], inplace=True)
    if "ret_bps" in df.columns:
        df["ret_bps"] = pd.to_numeric(df["ret_bps"], errors="coerce")
        df["ret"] = df["ret_bps"] / 10000.0
        
    print(f"Benchmark Ready: {len(df):,} rows.")
    return df

def load_factors():
    """Loads all 6 Fama-French Factor files into a dictionary."""
    print("Loading Factors...")
    factors = {}
    files_map = {
        ("Daily", "Global"):      "Factors_Daily_Global.txt",
        ("Daily", "Country"):     "Factors_Daily_Country.txt",
        ("Daily", "Replication"): "Factors_Daily_Replication.txt",
        ("Annual", "Global"):     "Factors_Annual_Global.txt",
        ("Annual", "Country"):    "Factors_Annual_Country.txt",
        ("Annual", "Replication"): "Factors_Annual_Replication.txt",
    }
    
    for (rebal, scope), fname in files_map.items():
        fpath = FACTOR_DIR / fname
        if fpath.exists():
            dtype_map = {"Country": str} if scope == "Country" else {}
            df = pd.read_csv(fpath, sep=SEP, dtype=dtype_map, low_memory=False)
            df["DayDate"] = pd.to_datetime(df["DayDate"], errors="coerce")
            factors[(rebal, scope)] = df
    print(f"Loaded {len(factors)} factor files.")
    return factors

def calc_deciles_vectorized(series):
    """
    Computes Deciles 1-10 using pd.qcut (True Statistical Method).
    """
    valid = series.dropna()
    if len(valid) < 2: 
        return pd.Series(index=series.index, dtype=float)

    try:
        # TRUE METHOD: Quantile Cut
        # duplicates='drop' merges bins if data is too clustered or sparse
        deciles = pd.qcut(valid, 10, labels=False, duplicates='drop') + 1
        
    except ValueError:
        # Fallback for extreme edge cases
        ranks = valid.rank(method='first')
        deciles = pd.qcut(ranks, 10, labels=False) + 1

    return deciles.astype(float)

def track_ram_usage(df):
    """Returns the deep memory usage of a DataFrame in GB."""
    mem_bytes = df.memory_usage(deep=True).sum()
    return mem_bytes / (1024**3)

# =============================================================================
# 3. WORKER FUNCTION (THE PIPELINE)
# =============================================================================

def process_pipeline_worker(input_path, df_bench_ref, factors_dict):
    """
    The Worker Logic. Runs entire lifecycle for one anomaly file in RAM.
    """
    os.environ["OMP_NUM_THREADS"] = "1"
    try:
        import numexpr
        numexpr.set_num_threads(1)
    except:
        pass

    stem = input_path.stem.split("_")[0].lower()
    start_t = time.time()
    
    try:
        # --- STEP 1: LOAD & MERGE ---
        dtype_map = {col: str for col in STRING_COLS if col in STRING_COLS}
        try:
            df_anom = pd.read_csv(input_path, sep=SEP, dtype=dtype_map, low_memory=False)
        except:
            df_anom = pd.read_csv(input_path, sep=SEP, low_memory=False)

        if "PIT Date" in df_anom.columns:
            df_anom["PIT Date"] = pd.to_datetime(df_anom["PIT Date"], errors="coerce")
            df_anom = df_anom.rename(columns={"PIT Date": "DayDate"})
        
        valid_ids = df_anom["ID"].unique()
        df_merged = df_bench_ref[df_bench_ref["ID"].isin(valid_ids)].copy()
        
        df_merged = pd.merge(df_merged, df_anom, on=["ID", "DayDate"], how="left", suffixes=("", "_new"))
        
        # --- STEP 2: FILL FORWARD ---
        df_merged = df_merged.sort_values(by=["ID", "DayDate"])
        df_merged["UpdateDate"] = np.where(df_merged[stem].notna(), 1, 0)

        # =========================================================================
        # SAVE INTERMEDIATE "INFORMATION ARRIVAL" DATASET (PARQUET)
        # =========================================================================
        # Goal: Save only the rows where new information arrived (UpdateDate == 1) 
        # for use in the "Information Lag" analysis (Table II).
        # =========================================================================
        
        mask_new_info = df_merged["UpdateDate"] == 1
        
        if mask_new_info.any():
            # Select only the columns required for the next script
            cols_to_save = ["ID", "DayDate", "UpdateDate", "FiscalPeriod"]
            valid_cols = [c for c in cols_to_save if c in df_merged.columns]
            
            df_intermediate = df_merged.loc[mask_new_info, valid_cols].copy()
            
            # Save as Parquet with a clear, descriptive name
            inter_fname = f"{stem}_Information_Arrival.parquet"
            inter_path = OUTPUT_DIR / inter_fname  # Or INTERMEDIATE_DIR if you defined one
            
            df_intermediate.to_parquet(inter_path, index=False)
            
            # Clean up memory
            del df_intermediate

        df_merged["_spell_id"] = df_merged.groupby("ID")["UpdateDate"].cumsum()
        
        cols_to_fill = [stem, "FiscalPeriod", "HistCurrency"]
        df_merged[cols_to_fill] = df_merged.groupby(["ID", "_spell_id"])[cols_to_fill].ffill()
        
        df_merged["_anchor_date"] = np.where(df_merged["UpdateDate"] == 1, df_merged["DayDate"], pd.NaT)
        df_merged["_anchor_date"] = pd.to_datetime(df_merged["_anchor_date"])
        df_merged["_anchor_date"] = df_merged.groupby(["ID", "_spell_id"])["_anchor_date"].ffill()
        df_merged["EventDate"] = df_merged.groupby(["ID", "_spell_id"]).cumcount()

        # --- STEP 3: CONSTRAINTS ---
        days_diff = (df_merged["DayDate"] - df_merged["_anchor_date"]).dt.days
        mask_too_old = (days_diff > TWO_YEARS_DAYS) | days_diff.isna()
        
        prev_dates = df_merged["DayDate"].shift(1)
        prev_ids = df_merged["ID"].shift(1)
        gap_days = (df_merged["DayDate"] - prev_dates).dt.days
        gap_days = np.where(df_merged["ID"] == prev_ids, gap_days, 0)
        mask_gap_error = gap_days > 14
        
        mask_invalid = mask_too_old | mask_gap_error
        df_merged.loc[mask_invalid, cols_to_fill] = np.nan
        df_merged = df_merged[df_merged[stem].notna()]
        
        df_merged.drop(columns=["_spell_id", "_anchor_date", "UpdateDate"], inplace=True)
        
        # RAM Safety Check
        peak_ram = track_ram_usage(df_merged)
        if peak_ram > 50: print(f"[{stem}] HIGH RAM USE: {peak_ram:.2f} GB")
        
        if "Country" in df_merged.columns:
            df_merged["_country_num"] = pd.to_numeric(df_merged["Country"], errors="coerce")
        df_merged["_rank_val"] = pd.to_numeric(df_merged[stem], errors="coerce")

        # --- STEP 4: DAILY DECILES ---
        df_merged["Decile_Total_Daily"] = np.nan
        df_merged["Decile_Country_Daily"] = np.nan
        df_merged["Decile_Rep_Daily"] = np.nan

        mask_rebal_daily = (df_merged["EventDate"] == 1) & (df_merged["_rank_val"].notna())
        
        if mask_rebal_daily.any():
            df_merged.loc[mask_rebal_daily, "Decile_Total_Daily"] = (
                df_merged[mask_rebal_daily].groupby("DayDate")["_rank_val"]
                .transform(calc_deciles_vectorized)
            )
            df_merged.loc[mask_rebal_daily, "Decile_Country_Daily"] = (
                df_merged[mask_rebal_daily].groupby(["DayDate", "Country"])["_rank_val"]
                .transform(calc_deciles_vectorized)
            )
            mask_rep = mask_rebal_daily & (df_merged["_country_num"].isin(REPLICATION_COUNTRIES))
            if mask_rep.any():
                df_merged.loc[mask_rep, "Decile_Rep_Daily"] = (
                    df_merged[mask_rep].groupby("DayDate")["_rank_val"]
                    .transform(calc_deciles_vectorized)
                )
        
        daily_cols = ["Decile_Total_Daily", "Decile_Country_Daily", "Decile_Rep_Daily"]
        df_merged[daily_cols] = df_merged.groupby("ID")[daily_cols].ffill()

        # --- STEP 5: ANNUAL DECILES ---
        df_merged["Decile_Total_Annual"] = np.nan
        df_merged["Decile_Country_Annual"] = np.nan
        df_merged["Decile_Rep_Annual"] = np.nan

        df_merged["_rebal_cohort"] = df_merged["DayDate"].dt.year.astype("int64")
        mask_prev = (df_merged["DayDate"].dt.month < 6) | ((df_merged["DayDate"].dt.month == 6) & (df_merged["DayDate"].dt.day < 30))
        df_merged.loc[mask_prev, "_rebal_cohort"] -= 1
        
        df_merged["_is_anchor"] = ~df_merged.duplicated(subset=["ID", "_rebal_cohort"], keep="first")
        mask_calc_annual = df_merged["_is_anchor"] & df_merged["_rank_val"].notna()
        
        if mask_calc_annual.any():
            df_merged.loc[mask_calc_annual, "Decile_Total_Annual"] = (
                df_merged[mask_calc_annual].groupby("DayDate")["_rank_val"]
                .transform(calc_deciles_vectorized)
            )
            df_merged.loc[mask_calc_annual, "Decile_Country_Annual"] = (
                df_merged[mask_calc_annual].groupby(["DayDate", "Country"])["_rank_val"]
                .transform(calc_deciles_vectorized)
            )
            mask_rep = mask_calc_annual & (df_merged["_country_num"].isin(REPLICATION_COUNTRIES))
            if mask_rep.any():
                df_merged.loc[mask_rep, "Decile_Rep_Annual"] = (
                    df_merged[mask_rep].groupby("DayDate")["_rank_val"]
                    .transform(calc_deciles_vectorized)
                )

        ann_cols = ["Decile_Total_Annual", "Decile_Country_Annual", "Decile_Rep_Annual"]
        df_merged[ann_cols] = df_merged.groupby(["ID", "_rebal_cohort"])[ann_cols].ffill()

        # --- STEP 6: SAVE LOOP ---
        files_created = []
        scenarios = [
            ("Daily", "Decile_Total_Daily",   "Global",      ["DayDate"]),
            ("Daily", "Decile_Country_Daily", "Country",     ["DayDate", "Country"]),
            ("Daily", "Decile_Rep_Daily",     "Replication", ["DayDate"]),
            ("Annual", "Decile_Total_Annual",   "Global",      ["DayDate"]),
            ("Annual", "Decile_Country_Annual", "Country",     ["DayDate", "Country"]),
            ("Annual", "Decile_Rep_Annual",     "Replication", ["DayDate"]),
        ]

        for rebal_mode, decile_col, scope, merge_keys in scenarios:
            mask_keep = df_merged[decile_col].isin([1, 10])
            if not mask_keep.any(): continue
                
            df_sub = df_merged[mask_keep].copy()
            df_sub.rename(columns={decile_col: "Decile"}, inplace=True)
            
            factor_key = (rebal_mode, scope)
            if factor_key not in factors_dict: continue
            df_factors = factors_dict[factor_key]
            
            # Type safety
            if "Country" in merge_keys:
                df_sub["Country"] = df_sub["Country"].astype(str)
                df_factors["Country"] = df_factors["Country"].astype(str)
            
            df_final = pd.merge(df_sub, df_factors, on=merge_keys, how="inner")
            
            if df_final.empty: continue
            
            # Clean columns
            cols_to_drop = [
                "_rank_val", "_country_num", "_rebal_cohort", "_is_anchor",
                "Decile_Total_Daily", "Decile_Country_Daily", "Decile_Rep_Daily",
                "Decile_Total_Annual", "Decile_Country_Annual", "Decile_Rep_Annual",
                "PCUR", "MV_LC", "ret_bps", "FiscalPeriod", "HistCurrency"
            ]
            df_final.drop(columns=[c for c in cols_to_drop if c in df_final.columns], inplace=True)
            
            out_name = f"{stem}_{scope}_{rebal_mode}_clean.parquet"
            df_final.to_parquet(OUTPUT_DIR / out_name, index=False)
            files_created.append(out_name)
            
            del df_sub, df_final
            
        elapsed = time.time() - start_t
        return f"DONE: {stem} -> {len(files_created)} files. Time: {elapsed/60:.1f} min"

    except Exception as e:
        return f"ERROR: {stem} - {str(e)}"

# =============================================================================
# 4. MAIN EXECUTION & VALIDATION
# =============================================================================

if __name__ == "__main__":
    total_start = time.time()
    
    # --- PART A: LOAD DATA ---
    df_bench = load_benchmark()
    factors_dict = load_factors()
    
    if df_bench.empty or not factors_dict:
        print("Critical Error: Missing Inputs. Stopping.")
        sys.exit()

    # --- PART B: PROCESSING ---
    all_files = sorted(list(INPUT_DIR.glob(INPUT_PATTERN)))
    print(f"\nProcessing {len(all_files)} anomalies (Full Production Run).")
    print(f"Workers: {N_JOBS}")
    print("Output Format: Parquet")
    print("-" * 60)
    
    results = Parallel(n_jobs=N_JOBS, backend="loky", verbose=10)(
        delayed(process_pipeline_worker)(f, df_bench, factors_dict) for f in all_files
    )
    
    print("\n" + "="*60)
    for res in results:
        print(res)
    print("="*60)
    print(f"Total Pipeline Time: {(time.time() - total_start)/60:.1f} minutes.")
    
    del results
    gc.collect()

    # --- PART C: FINAL VALIDATION ---
    print("\n" + "*"*60)
    print(f"5. VALIDATION CHECK FOR ID: {VAL_ID}")
    print(f"   Target Window: {VAL_START} to {VAL_END}")
    print("*"*60 + "\n")

    # 1. INPUT CHECK
    print(f"A. RAW INPUT ({VAL_ANOMALY})")
    # Smart file search (Case Insensitive)
    input_file_path = next((f for f in all_files if f.name.lower().startswith(VAL_ANOMALY.lower() + "_")), None)
    
    if input_file_path:
        try:
            df_in = pd.read_csv(input_file_path, sep=SEP, dtype={"ID": str}, low_memory=False)
            if "PIT Date" in df_in.columns: df_in.rename(columns={"PIT Date": "DayDate"}, inplace=True)
            df_in["DayDate"] = pd.to_datetime(df_in["DayDate"])
            
            mask_win = (df_in["ID"] == VAL_ID) & (df_in["DayDate"].between(VAL_START, VAL_END))
            view = df_in[mask_win]
            
            if not view.empty:
                print(view[["ID", "DayDate", VAL_ANOMALY]].to_string(index=False))
            else:
                print("   (No data in window. Checking fill-forward source...)")
                mask_prior = (df_in["ID"] == VAL_ID) & (df_in["DayDate"] < VAL_START)
                prior = df_in[mask_prior].sort_values("DayDate").tail(1)
                if not prior.empty:
                    print(prior[["ID", "DayDate", VAL_ANOMALY]].to_string(index=False))
                else:
                    print("   (No data found for this ID at all.)")
        except Exception as e:
            print(f"   Error: {e}")
    else:
        print("   (Input file not found)")
    print("-" * 40)

    # 2. OUTPUT CHECK (ALL 6 SCENARIOS)
    print(f"C. FINAL OUTPUTS (Checking ALL 6 Scenarios for {VAL_ANOMALY})")
    
    val_scenarios = [
        ("Daily", "Global"), ("Daily", "Country"), ("Daily", "Replication"),
        ("Annual", "Global"), ("Annual", "Country"), ("Annual", "Replication")
    ]

    for rebal, scope in val_scenarios:
        out_fname = f"{VAL_ANOMALY}_{scope}_{rebal}_clean.parquet"
        out_path = OUTPUT_DIR / out_fname
        print(f"\n   >>> Checking: {out_fname}")
        
        if out_path.exists():
            try:
                df_out = pd.read_parquet(out_path)
                df_out["DayDate"] = pd.to_datetime(df_out["DayDate"])
                
                mask_out = (df_out["ID"] == VAL_ID) & (df_out["DayDate"].between(VAL_START, VAL_END))
                view_out = df_out[mask_out]
                
                if not view_out.empty:
                    # Dynamically find columns to show (Fixes duplicate column bugs)
                    possible_cols = ["ID", "DayDate", VAL_ANOMALY, "acc", "ACC", "Decile", "ret", 
                                     "Mkt-RF", "SMB", "HML", "RMW", "CMA", "Mom", "RF"]
                    cols = [c for c in possible_cols if c in df_out.columns]
                    # Remove duplicates from list if any exist
                    cols = list(dict.fromkeys(cols)) 
                    print(view_out[cols].to_string(index=False))
                else:
                    print("       (No rows. Decile likely not 1 or 10, or filtered by gap rules.)")
            except Exception as e:
                print(f"       Error reading file: {e}")
        else:
            print("       (File not found. Pipeline may have skipped it if empty.)")

    # Cleanup
    del df_bench
    del factors_dict
    gc.collect()

Loading Benchmark: FF_Benchmark_data_clean.txt...
Filtering Benchmark for range: 1994-01-01 to 2024-12-31
Benchmark Ready: 143,802,215 rows.
Loading Factors...
Loaded 6 factor files.

Processing 27 anomalies (Full Production Run).
Workers: 6
Output Format: Parquet
------------------------------------------------------------


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed: 11.2min
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed: 17.9min
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed: 41.0min
[Parallel(n_jobs=6)]: Done  19 out of  27 | elapsed: 55.8min remaining: 23.5min
[Parallel(n_jobs=6)]: Done  22 out of  27 | elapsed: 60.2min remaining: 13.7min
[Parallel(n_jobs=6)]: Done  25 out of  27 | elapsed: 67.9min remaining:  5.4min
[Parallel(n_jobs=6)]: Done  27 out of  27 | elapsed: 74.1min finished



DONE: acc -> 6 files. Time: 11.6 min
DONE: ag -> 6 files. Time: 16.9 min
DONE: at -> 6 files. Time: 14.7 min
DONE: cat -> 6 files. Time: 14.0 min
DONE: cpm -> 6 files. Time: 15.0 min
DONE: ec -> 6 files. Time: 8.9 min
DONE: es -> 6 files. Time: 13.2 min
DONE: gp -> 6 files. Time: 15.2 min
DONE: ig -> 6 files. Time: 16.1 min
DONE: inv -> 6 files. Time: 13.9 min
DONE: ltg -> 6 files. Time: 9.7 min
DONE: nca -> 6 files. Time: 15.7 min
DONE: noa -> 6 files. Time: 16.1 min
DONE: nwc -> 6 files. Time: 16.0 min
DONE: ol -> 6 files. Time: 12.8 min
DONE: osc -> 6 files. Time: 15.5 min
DONE: pm -> 6 files. Time: 15.6 min
DONE: poa -> 6 files. Time: 15.3 min
DONE: pro -> 6 files. Time: 15.3 min
DONE: pta -> 6 files. Time: 14.2 min
DONE: roe -> 6 files. Time: 14.5 min
DONE: rs -> 6 files. Time: 13.8 min
DONE: sg -> 6 files. Time: 15.4 min
DONE: sli -> 6 files. Time: 13.5 min
DONE: slx -> 6 files. Time: 13.0 min
DONE: txf -> 6 files. Time: 11.1 min
DONE: tx -> 6 files. Time: 15.5 min
Total Pipelin

# Event Time Approach

### Event Time Regression TBD

=> Comment: RunTime Error Warning is not a problem as it is handled.

In [4]:
import os
import sys
import time
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from pathlib import Path
from linearmodels.panel import PanelOLS
import gc

# =================================================================================================
# MASTER SCRIPT: SEQUENTIAL EVENT-TIME ANOMALY REGRESSION (FULL PRODUCTION)
# =================================================================================================
#
# 1. EXECUTIVE SUMMARY
# --------------------
# This script performs a robust financial analysis of 27 equity market anomalies. It calculates
# "Event-Time Alphas" by running Long-Short regressions on daily data. The analysis is performed
# across three distinct scopes:
#    a) Total (Global Universe)
#    b) Replication (USA & Canada / ID 124 & 840)
#    c) Individual Countries (Iterative loop)
#
# 2. OUTPUT FORMATTING (UPDATED)
# ------------------------------
#    * "Average_Overview" Tab:
#      - Now provides a detailed granular view for the "Average Anomaly" across ALL time windows.
#      - Rows: Country + Time Window (All, 20, 40, 80, 81-240).
#      - Metrics: Annualized Alpha, SE, T-Stat, and Significance Stars.
#
#    * Summary Tabs (Total, Rep, Country):
#      - Features "Interleaved Columns" for easier reading of statistical significance.
#      - Format: [Window Alpha] | [Window T-Stat] | [Next Window Alpha] | [Next Window T-Stat] ...
#      - Rows: Each individual anomaly + the "AVERAGE" anomaly at the top.
#
# 3. SYSTEM ARCHITECTURE (STABILITY FOCUSED)
# ------------------------------------------
#    * SEQUENTIAL EXECUTION: The script runs on a SINGLE WORKER (Main Process).
#      - Reason: To strictly avoid "Resource temporarily unavailable" / "pthread_create" errors 
#        caused by OpenBLAS thread explosion on restricted servers (e.g., Binder/JupyterHub).
#    * THREAD LOCKING: Environment variables (OMP_NUM_THREADS, etc.) are forced to '1' before
#      any math libraries load.
#    * MEMORY MANAGEMENT: Garbage collection (gc.collect) is triggered after every anomaly file 
#      to prevent RAM bloat during the long overnight run.
#
# 4. ECONOMETRIC SPECIFICATION
# ----------------------------
#    * MODEL: 5-Factor Fama-French + Momentum (6 Factors total).
#      Formula: eret ~ 1 + hedge * (mktrf + smb + rmw + cma + umd)
#    * ESTIMATOR (PANEL): Linearmodels 'PanelOLS' used for individual anomalies.
#      - Covariance: Clustered by Entity (Firm) and Time (Date).
#    * ESTIMATOR (AVERAGE): Statsmodels 'OLS' used for the "Average Anomaly".
#      - Covariance: Newey-West HAC (Heteroskedasticity and Autocorrelation Consistent).
#      - Lags: 6 Trading Days (approx. 1 week lag).
#    * ANNUALIZATION:
#      - Coefficients: Geometric Compounding -> ((1 + daily_alpha) ^ 252) - 1
#      - Standard Errors: Delta Method Approximation -> SE * 252 * (1 + daily_alpha)^251
#
# 5. DATA PIPELINE & FILTERS
# --------------------------
#    * INPUT: Parquet files named by anomaly stem (e.g., "acc_Country_Daily_clean.parquet").
#    * SCOPE HANDLING:
#      - "Total": Runs regression on the full dataset.
#      - "Replication": automatically filters for USA (840) and Canada (124).
#      - "Country": Loops through every country code found in the file.
#    * SAFETY FILTER (CRITICAL):
#      - Countries with fewer than 30 unique Firm IDs are DROPPED automatically.
#      - This prevents Singular Matrix errors and statistical noise from tiny markets.
#    * EVENT WINDOWS:
#      - All (Full history), 20 days, 40 days, 80 days, and 81-240 days.
#
# 6. AUDITING & LOGGING
# ---------------------

# --- THREAD SAFETY (CRITICAL) ---
os.environ["NUMEXPR_MAX_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
try:
    import numexpr
    numexpr.set_num_threads(1)
except: pass

# =================================================================================================
# 1. CONFIGURATION
# =================================================================================================

# --- PATHS ---
DATA_FOLDER = Path(Temp_file_path_R) 
OUTPUT_EXCEL = os.path.join(DATA_FOLDER, "EventTimeRegression_Daily_Final_FULL.xlsx")

# --- ANOMALY CONFIGURATION (ALL 27 ITEMS) ---
ANOMALY_CONFIG = {
    "acc": (1, 10), "ag": (1, 10), "at": (10, 1), "cat": (10, 1),
    "cpm": (10, 1), "ec": (10, 1), "es": (10, 1), "gp": (10, 1),
    "ig": (1, 10), "inv": (1, 10), "ltg": (10, 1), "nca": (1, 10),
    "noa": (1, 10), "nwc": (1, 10), "ol": (10, 1), "osc": (1, 10),
    "pm": (10, 1), "poa": (1, 10), "pro": (10, 1), "pta": (1, 10),
    "roe": (10, 1), "rs": (10, 1), "sg": (1, 10), "sli": (10, 1),
    "slx": (10, 1), "tx": (10, 1), "txf": (1, 10),
}

# --- FILE PATTERNS ---
FILE_PATTERNS = {
    "Total":       "{stem}_Global_Daily_clean.parquet",       
    "Replication": "{stem}_Replication_Daily_clean.parquet", 
    "Country":     "{stem}_Country_Daily_clean.parquet"      
}

# --- REGRESSION PARAMETERS ---
TRADING_DAYS = 252
MIN_OBS = 252
NEWEY_WEST_LAGS = 6
MIN_COUNTRY_IDS = 30     # Filter for small countries

# --- EVENT WINDOWS (Excludes Day 0) ---
WINDOW_SPECS = {
    # "All"
    "All":     lambda s: s >= 1,  
    
    # Strict 1 to X windows
    "20":      lambda s: (s >= 1) & (s <= 20),
    "40":      lambda s: (s >= 1) & (s <= 40),
    "80":      lambda s: (s >= 1) & (s <= 80),
    
    # Distinct bucket for the tail
    "81_240":  lambda s: (s >= 81) & (s <= 240),
}

# --- MODEL SPECIFICATION ---
FORMULA = "eret ~ 1 + hedge * (mktrf + smb + rmw + cma + umd)"
FACTOR_COLS = ["mktrf", "smb", "rmw", "cma", "umd"]

# --- DTYPES ---
DTYPE_DICT = {
    "ret": "float32", "rf": "float32", 
    "mktrf": "float32", "smb": "float32", "rmw": "float32", 
    "cma": "float32", "umd": "float32", 
}

# =================================================================================================
# 2. HELPER FUNCTIONS
# =================================================================================================

def add_stars(t):
    at = abs(t)
    if at >= 2.58: return "***"
    elif at >= 1.96: return "**"
    elif at >= 1.65: return "*"
    return ""

def extract_stats(result, label, window, scope, country_lbl, model_type="Panel"):
    rows = []
    if model_type == "Panel":
        params = result.params
        std = result.std_errors
    else: # Statsmodels
        params = result.params
        std = result.bse
        
    for pname in params.index:
        if "hedge" in pname and ":" not in pname: 
            coef = params[pname]
            se = std[pname]
            
            ann_coef = (1.0 + coef) ** TRADING_DAYS - 1.0
            derivative = TRADING_DAYS * (1.0 + coef) ** (TRADING_DAYS - 1)
            ann_se = se * derivative
            
            if model_type == "Panel": tstat = coef / se if se != 0 else 0
            else: tstat = result.tvalues[pname]

            rows.append({
                "file": label, "window": window, "scope": scope, "country": country_lbl,
                "param": "hedge",
                "annualized_alpha": ann_coef, "annualized_se": ann_se,
                "daily_alpha": coef, "daily_se": se,
                "tstat": tstat, "stars": add_stars(tstat)
            })
            break 
    return rows

def calc_daily_metrics(df, label):
    if df.empty: return None
    ret_grp = df.groupby(["DayDate", "hedge"])["eret"].mean().unstack()
    if 1.0 in ret_grp.columns: ret_grp.rename(columns={1.0: "long_ret"}, inplace=True)
    if 0.0 in ret_grp.columns: ret_grp.rename(columns={0.0: "short_ret"}, inplace=True)
    
    fact_grp = df.groupby("DayDate")[FACTOR_COLS].mean()
    daily_data = ret_grp.join(fact_grp, how="inner")
    
    if "long_ret" not in daily_data.columns and "short_ret" not in daily_data.columns: return None
    return daily_data

# =================================================================================================
# 3. PROCESSING LOGIC
# =================================================================================================

def process_anomaly_stem(stem, long_dec, short_dec, current_idx, total_count):
    """Main Logic Function with Auditing."""
    print(f"\n[{current_idx}/{total_count}] Processing Anomaly: {stem} ...")
    results_rows = []
    return_series_dict = {}

    def process_one_file(file_type, file_pattern, scope_name_func):
        fpath = DATA_FOLDER / file_pattern.format(stem=stem)
        
        # --- AUDIT: CHECK FILE ---
        if not fpath.exists(): 
            print(f"   [Skip] File missing: {fpath.name}")
            return
        
        try:
            df = pd.read_parquet(fpath)
            
            # --- AUDIT: CHECK CONTENT ---
            if file_type == "Country":
                if "Country" in df.columns:
                    raw_ctry_count = df["Country"].astype(str).nunique()
                    print(f"   [AUDIT] {file_type} File: Loaded {len(df):,} rows covering {raw_ctry_count} countries.")
            
            # RENAME & CAST
            rename_map = {
                "Mkt-RF": "mktrf", "SMB": "smb", "HML": "hml", 
                "RMW": "rmw", "CMA": "cma", "Mom": "umd", "RF": "rf", "Rf": "rf" 
            }
            df.rename(columns=rename_map, inplace=True)
            for col, dtype in DTYPE_DICT.items():
                if col in df.columns: df[col] = df[col].astype(dtype)

            # RETURNS & HEDGE
            if "Decile" in df.columns: dcol = "Decile"
            else: dcol = f"Decile_{file_type}" 
            
            if "ret" not in df.columns and "ret_bps" in df.columns:
                 df["ret"] = df["ret_bps"] / 10000.0
            if "rf" not in df.columns: df["rf"] = 0.0
            df["eret"] = df["ret"] - df["rf"]
            
            df["hedge"] = np.nan
            df.loc[df[dcol] == long_dec, "hedge"] = 1
            df.loc[df[dcol] == short_dec, "hedge"] = 0
            
            # FILTER
            valid_factors = [f for f in FACTOR_COLS if f in df.columns]
            df = df.dropna(subset=["hedge", "eret"] + valid_factors).copy()
            df = df.set_index(["ID", "DayDate"]).sort_index()

            # SCOPES & COUNTRY FILTER
            iter_items = [(scope_name_func(None), df)]
            
            if file_type == "Country":
                iter_items = []
                dropped_list = []
                keep_count = 0
                unique_countries = df["Country"].unique()
                
                for c in unique_countries:
                    sub_c = df[df["Country"]==c]
                    # Check Index Level 0 ("ID") for unique counts
                    unique_ids = sub_c.index.get_level_values("ID").nunique()
                    
                    if unique_ids >= MIN_COUNTRY_IDS:
                        iter_items.append((str(c), sub_c))
                        keep_count += 1
                    else:
                        dropped_list.append(f"{c}({unique_ids})")
                
                print(f"   [FILTER] Kept {keep_count} countries. Dropped {len(dropped_list)} (ID < {MIN_COUNTRY_IDS}).")
            
            # REGRESSIONS
            for country_lbl, sub_df in iter_items:
                if sub_df.empty: continue
                scope_key = country_lbl if file_type == "Country" else file_type
                if scope_key not in return_series_dict: return_series_dict[scope_key] = {}

                for w_name, w_cond in WINDOW_SPECS.items():
                    mask = w_cond(sub_df["EventDate"])
                    sub_w = sub_df[mask]
                    
                    if not sub_w.empty:
                        metrics = calc_daily_metrics(sub_w, stem)
                        if metrics is not None:
                            if w_name not in return_series_dict[scope_key]:
                                return_series_dict[scope_key][w_name] = []
                            return_series_dict[scope_key][w_name].append(metrics)

                    if len(sub_w) > MIN_OBS and sub_w["hedge"].nunique() > 1:
                        avail_formula = "eret ~ 1 + hedge * (" + " + ".join(valid_factors) + ")"
                        try:
                            res = PanelOLS.from_formula(avail_formula, sub_w).fit(cov_type="clustered", cluster_entity=True, cluster_time=True)
                            results_rows.extend(extract_stats(res, stem, w_name, file_type, country_lbl, "Panel"))
                        except: pass

            del df; gc.collect()
            
        except Exception as e:
            print(f"   [Error] Processing {file_type} for {stem}: {e}")

    process_one_file("Total", FILE_PATTERNS["Total"], lambda x: "Total")
    process_one_file("Replication", FILE_PATTERNS["Replication"], lambda x: "124_840")
    process_one_file("Country", FILE_PATTERNS["Country"], lambda x: "Country")

    return {"rows": results_rows, "series": return_series_dict}

# =================================================================================================
# 4. MAIN EXECUTION
# =================================================================================================

if __name__ == "__main__":
    start_time = time.time()
    total_anomalies = len(ANOMALY_CONFIG)
    
    print("="*80)
    print(f"STARTING SEQUENTIAL REGRESSION RUN")
    print(f"Anomalies: {total_anomalies}")
    print(f"Output: {OUTPUT_EXCEL}")
    print("="*80)
    
    # --- STEP 1: SEQUENTIAL LOOP ---
    results = []
    for i, (stem, ls) in enumerate(ANOMALY_CONFIG.items(), 1):
        res = process_anomaly_stem(stem, ls[0], ls[1], i, total_anomalies)
        results.append(res)
    
    # --- STEP 2: AGGREGATE ---
    print("\n" + "="*80)
    print("AGGREGATING DATA & CALCULATING AVERAGES")
    print("="*80)
    
    all_rows = []
    avg_inputs = {}

    for r in results:
        all_rows.extend(r["rows"])
        for scope_key, window_dict in r["series"].items():
            if scope_key not in avg_inputs: avg_inputs[scope_key] = {}
            for w_name, df_list in window_dict.items():
                if w_name not in avg_inputs[scope_key]: avg_inputs[scope_key][w_name] = []
                avg_inputs[scope_key][w_name].extend(df_list)

    # --- STEP 3: CALCULATE "AVERAGE ANOMALY" ---
    avg_rows = []

    def solve_average_unified(scope_name, window_name, input_dfs, country_lbl):
        if not input_dfs: return []
        combined = pd.concat(input_dfs)
        daily_avgs = combined.groupby("DayDate").mean()
        
        if "long_ret" not in daily_avgs.columns or "short_ret" not in daily_avgs.columns: return []
        daily_avgs = daily_avgs.dropna(subset=["long_ret", "short_ret"])
        if daily_avgs.empty: return []

        longs = daily_avgs[["long_ret"] + [c for c in FACTOR_COLS if c in daily_avgs.columns]].copy()
        longs = longs.rename(columns={"long_ret": "eret"}); longs["hedge"] = 1
        shorts = daily_avgs[["short_ret"] + [c for c in FACTOR_COLS if c in daily_avgs.columns]].copy()
        shorts = shorts.rename(columns={"short_ret": "eret"}); shorts["hedge"] = 0
        
        reg_df = pd.concat([longs, shorts]).sort_index()
        
        try:
            avail_factors = [c for c in FACTOR_COLS if c in reg_df.columns]
            dyn_formula = "eret ~ 1 + hedge * (" + " + ".join(avail_factors) + ")"
            mod = smf.ols(dyn_formula, data=reg_df)
            res = mod.fit(cov_type='HAC', cov_kwds={'maxlags': NEWEY_WEST_LAGS})
            return extract_stats(res, "AVERAGE", window_name, scope_name, country_lbl, "Statsmodels")
        except: return []

    for scope_key, window_dict in avg_inputs.items():
        if scope_key == "Total": final_scope, final_country = "Total", "Total"
        elif scope_key == "Replication": final_scope, final_country = "Replication", "124_840"
        else: final_scope, final_country = "Country", scope_key

        for w_name, dfs in window_dict.items():
            avg_rows.extend(solve_average_unified(final_scope, w_name, dfs, final_country))

    final_df = pd.DataFrame(all_rows + avg_rows)

    # --- STEP 4: EXCEL REPORTING (FORMATTING UPDATES) ---
    print(f"\nWriting results to {OUTPUT_EXCEL}...")
    
    if not final_df.empty:
        found_countries = final_df[final_df["scope"]=="Country"]["country"].unique()
        print(f"Final Report contains data for {len(found_countries)} unique countries.")
        
        with pd.ExcelWriter(OUTPUT_EXCEL) as writer:
            
            # --- TAB 1: Average Overview (DETAILED: ALL WINDOWS) ---
            # Now includes ALL windows (All, 20, 40...) for every country
            avg_mask = (final_df["file"] == "AVERAGE") & (final_df["window"].isin(["All", "20", "40", "80", "81_240"]))
            overview = final_df[avg_mask].copy()
            if not overview.empty:
                overview["Alpha %"] = overview["annualized_alpha"] * 100
                overview["SE %"] = overview["annualized_se"] * 100
                
                # Sort logic to keep windows ordered
                win_order = {"All": 0, "20": 1, "40": 2, "80": 3, "81_240": 4}
                overview["w_rank"] = overview["window"].map(win_order)
                overview = overview.sort_values(["scope", "country", "w_rank"])
                
                # Columns to display
                cols = ["scope", "country", "window", "Alpha %", "SE %", "tstat", "stars"]
                overview[cols].to_excel(writer, sheet_name="Average_Overview", index=False)
            
            # --- HELPER: Interleaved Columns for Summary Tabs ---
            def write_summary_tab(df_in, sheet_name):
                if df_in.empty: return
                
                # Filter relevant windows
                windows = ["All", "20", "40", "80", "81_240"]
                df_in = df_in[df_in["window"].isin(windows)].copy()
                
                # Create Pre-Formatted Columns
                df_in["Alpha"] = (df_in["annualized_alpha"]*100).map('{:,.2f}'.format) + df_in["stars"]
                df_in["T-Stat"] = df_in["tstat"].map('{:,.2f}'.format)
                
                # Construct Interleaved Table Manually for Control
                # Row Index = File (Anomaly), Columns = Interleaved Windows
                unique_files = sorted(df_in["file"].unique())
                # Ensure AVERAGE is at top
                if "AVERAGE" in unique_files:
                    unique_files.remove("AVERAGE")
                    unique_files = ["AVERAGE"] + unique_files
                
                final_piv = pd.DataFrame(index=unique_files)
                
                for w in windows:
                    # Filter for specific window
                    sub = df_in[df_in["window"] == w]
                    if sub.empty: continue
                    
                    # Set index to file to match main DataFrame
                    sub = sub.set_index("file")
                    
                    # Add Columns
                    if "Alpha" in sub.columns:
                        final_piv[w] = sub["Alpha"]
                    if "T-Stat" in sub.columns:
                        final_piv[f"{w} (t-stat)"] = sub["T-Stat"]
                
                final_piv.to_excel(writer, sheet_name=sheet_name)

            # --- GENERATE TABS ---
            write_summary_tab(final_df[final_df["scope"]=="Total"], "Summary_Total")
            write_summary_tab(final_df[final_df["scope"]=="Replication"], "Summary_Rep")
            
            countries = final_df[final_df["scope"]=="Country"]["country"].unique()
            for c in sorted(countries):
                sheet_name = f"Summary_C{c}"[:31]
                sub_c = final_df[(final_df["scope"]=="Country") & (final_df["country"]==c)]
                write_summary_tab(sub_c, sheet_name)
            
            # Raw Data
            final_df.to_excel(writer, sheet_name="Raw_Results", index=False)
            
        print("Success! File saved.")
    else:
        print("Warning: No results generated.")

    elapsed = (time.time() - start_time) / 60
    print(f"\nCompleted in {elapsed:.1f} minutes.")

STARTING SEQUENTIAL REGRESSION RUN
Anomalies: 27
Output: /home/jovyan/work/hpool1/pseidel/test/Temp/TempRegressionModel/EventTimeRegression_Daily_Final_FULL.xlsx

[1/27] Processing Anomaly: acc ...
   [AUDIT] Country File: Loaded 27,766,736 rows covering 66 countries.
   [FILTER] Kept 51 countries. Dropped 11 (ID < 30).


/home/jovyan/.local/lib/python3.12/site-packages/linearmodels/panel/results.py:88: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/jovyan/.local/lib/python3.12/site-packages/linearmodels/panel/results.py:88: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")



[2/27] Processing Anomaly: ag ...
   [AUDIT] Country File: Loaded 35,645,205 rows covering 66 countries.
   [FILTER] Kept 58 countries. Dropped 5 (ID < 30).


/home/jovyan/.local/lib/python3.12/site-packages/linearmodels/panel/results.py:88: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/jovyan/.local/lib/python3.12/site-packages/linearmodels/panel/results.py:88: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/jovyan/.local/lib/python3.12/site-packages/linearmodels/panel/results.py:88: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/jovyan/.local/lib/python3.12/site-packages/linearmodels/panel/results.py:88: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")



[3/27] Processing Anomaly: at ...
   [AUDIT] Country File: Loaded 36,466,055 rows covering 66 countries.
   [FILTER] Kept 55 countries. Dropped 8 (ID < 30).

[4/27] Processing Anomaly: cat ...
   [AUDIT] Country File: Loaded 33,613,889 rows covering 66 countries.
   [FILTER] Kept 55 countries. Dropped 8 (ID < 30).

[5/27] Processing Anomaly: cpm ...
   [AUDIT] Country File: Loaded 35,110,426 rows covering 66 countries.
   [FILTER] Kept 57 countries. Dropped 6 (ID < 30).


...


[24/27] Processing Anomaly: sli ...
   [AUDIT] Country File: Loaded 33,901,518 rows covering 66 countries.
   [FILTER] Kept 56 countries. Dropped 7 (ID < 30).

[25/27] Processing Anomaly: slx ...
   [AUDIT] Country File: Loaded 30,602,398 rows covering 66 countries.
   [FILTER] Kept 55 countries. Dropped 8 (ID < 30).

[26/27] Processing Anomaly: tx ...
   [AUDIT] Country File: Loaded 38,213,750 rows covering 66 countries.
   [FILTER] Kept 57 countries. Dropped 6 (ID < 30).


/home/jovyan/.local/lib/python3.12/site-packages/linearmodels/panel/results.py:88: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")



[27/27] Processing Anomaly: txf ...
   [AUDIT] Country File: Loaded 25,076,477 rows covering 66 countries.
   [FILTER] Kept 52 countries. Dropped 11 (ID < 30).


/home/jovyan/.local/lib/python3.12/site-packages/linearmodels/panel/results.py:88: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")



AGGREGATING DATA & CALCULATING AVERAGES

Writing results to /home/jovyan/work/hpool1/pseidel/test/Temp/TempRegressionModel/EventTimeRegression_Daily_Final_FULL.xlsx...
Final Report contains data for 58 unique countries.
Success! File saved.

Completed in 537.3 minutes.


# Calendar Event Time Approach

### Calendar Time (Not Rolling, Code Replication, Date Clustering)

=> Comment: RunTime Error Warning is not a problem as it is handled.

In [5]:
import os
import sys
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from pathlib import Path
import gc

# =================================================================================================
# MASTER SCRIPT: CALENDAR-TIME COMPARISON (MODEL 1: CLUSTERED ERRORS - FIXED)
# =================================================================================================

# --- 1. CONFIGURATION ---

try:
    DATA_FOLDER = Path(Temp_file_path_R)
except NameError:
    print("Error: 'Temp_file_path_R' variable is not defined.")
    sys.exit("Stopping execution because fallback path is disabled.")

OUTPUT_EXCEL = DATA_FOLDER / "CalendarTime_Model1_Clustered.xlsx"

os.environ["NUMEXPR_MAX_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

# ANOMALY LIST
ANOMALY_CONFIG = {
    "acc": (1, 10), "ag": (1, 10), "at": (10, 1), "cat": (10, 1),
    "cpm": (10, 1), "ec": (10, 1), "es": (10, 1), "gp": (10, 1),
    "ig": (1, 10), "inv": (1, 10), "ltg": (10, 1), "nca": (1, 10),
    "noa": (1, 10), "nwc": (1, 10), "ol": (10, 1), "osc": (1, 10),
    "pm": (10, 1), "poa": (1, 10), "pro": (10, 1), "pta": (1, 10),
    "roe": (10, 1), "rs": (10, 1), "sg": (1, 10), "sli": (10, 1),
    "slx": (10, 1), "tx": (10, 1), "txf": (1, 10),
}

# PATTERNS
PATTERNS = {
    "Total":       "{stem}_Global_{freq}_clean.parquet",
    "Replication": "{stem}_Replication_{freq}_clean.parquet",
    "Country":     "{stem}_Country_{freq}_clean.parquet"
}

# SETTINGS
INTERNAL_FACTORS = ["mktrf", "smb", "rmw", "cma", "umd"] 
TRADING_DAYS = 252
MIN_COUNTRY_IDS = 30

# =================================================================================================
# 2. HELPER FUNCTIONS
# =================================================================================================

def annualized_stats(params, bse, term_name):
    """Calculates Annualized Alpha and SE."""
    if term_name not in params: return np.nan, np.nan, ""
    
    coef = params[term_name]
    se = bse[term_name]
    
    if abs(coef) > 0.5: return np.nan, np.nan, " (Outlier)"

    try:
        ann_coef = (1 + coef)**TRADING_DAYS - 1
        derivative = TRADING_DAYS * (1 + coef)**(TRADING_DAYS - 1)
        ann_se = se * derivative
        
        tstat = coef / se if se != 0 else 0
        stars = ""
        if abs(tstat) >= 2.58: stars = "***"
        elif abs(tstat) >= 1.96: stars = "**"
        elif abs(tstat) >= 1.65: stars = "*"
        
        return ann_coef * 100, ann_se * 100, stars
        
    except (OverflowError, ValueError, RuntimeWarning):
        return np.nan, np.nan, " (Error)"

def run_clustered_regression(df, formula, cluster_col="DayDate"):
    """
    Runs OLS with Clustered SE. 
    Explicitly drops NaNs to align the cluster array with the regression data.
    """
    if df.empty or df['eret'].nunique() <= 1: return None
    
    # 1. Identify columns used in the formula (plus the cluster column)
    # We cheat slightly and just drop NaNs from ALL factors + eret + hedge + cluster_col
    # This prevents the 'shape mismatch' crash in statsmodels.
    cols_to_check = ['eret', 'hedge', cluster_col] + INTERNAL_FACTORS
    
    # Only check columns that actually exist in the DF (e.g. 'is_daily' might be in formula)
    cols_present = [c for c in cols_to_check if c in df.columns]
    
    # 2. Drop NaNs explicitly
    df_clean = df.dropna(subset=cols_present).copy()
    
    if df_clean.empty: return None

    try:
        mod = smf.ols(formula, data=df_clean)
        # 3. Pass the CLEAN dataframe column to cov_kwds
        res = mod.fit(cov_type='cluster', cov_kwds={'groups': df_clean[cluster_col]})
        return res
    except Exception as e:
        # print(f"Reg Error: {e}") # Uncomment for debug
        return None

# =================================================================================================
# 3. DATA LOADER
# =================================================================================================

def get_portfolio_data_optimized(stem, scope, freq, long_dec, short_dec):
    """Loads ONE file, aggregates to portfolio level."""
    fname = PATTERNS[scope].format(stem=stem, freq=freq)
    fpath = DATA_FOLDER / fname
    
    if not fpath.exists(): 
        print(f"      [Warning] File not found: {fname}")
        return None
    
    col_map = {"rf": "RF", "mktrf": "Mkt-RF", "smb": "SMB", "rmw": "RMW", "cma": "CMA", "umd": "Mom"}
    rename_map = {v: k for k, v in col_map.items()}
    file_cols_needed = ["DayDate", "Decile", "ret", col_map["rf"]] + [col_map[f] for f in INTERNAL_FACTORS]
    if scope == "Country": file_cols_needed += ["Country", "ID"]
    
    try:
        df = pd.read_parquet(fpath, columns=file_cols_needed)
        df.rename(columns=rename_map, inplace=True)
        
        df["hedge"] = np.nan
        df.loc[df["Decile"] == long_dec, "hedge"] = 1
        df.loc[df["Decile"] == short_dec, "hedge"] = 0
        df.dropna(subset=["hedge"], inplace=True)
        df["eret"] = df["ret"] - df["rf"]
        
        if scope == "Country":
            df["id_count"] = df.groupby("Country")["ID"].transform("nunique")
            df = df[df["id_count"] >= MIN_COUNTRY_IDS].drop(columns=["id_count", "ID"])
            
        grp_cols = ["DayDate", "hedge"]
        if "Country" in df.columns: grp_cols.append("Country")
        
        port_df = df.groupby(grp_cols)[["eret"] + INTERNAL_FACTORS].mean().reset_index()
        return port_df
        
    except Exception as e:
        print(f"      [Error] Failed reading {fname}: {e}")
        return None

# =================================================================================================
# 4. MAIN LOGIC
# =================================================================================================

def analyze_scope_safe(scope_name):
    print(f"\n" + "="*60)
    print(f"STARTING SCOPE: {scope_name}")
    print("="*60)
    
    final_results = []
    avg_cache_daily = {}
    avg_cache_annual = {}
    
    for i, (anom, (l_dec, s_dec)) in enumerate(ANOMALY_CONFIG.items(), 1):
        print(f"[{i}/{len(ANOMALY_CONFIG)}] Processing Anomaly: {anom.upper()} ...")
        
        df_d = get_portfolio_data_optimized(anom, scope_name, "Daily", l_dec, s_dec)
        df_a = get_portfolio_data_optimized(anom, scope_name, "Annual", l_dec, s_dec)
        gc.collect() 
        
        if df_d is None or df_a is None: continue

        iter_items = []
        if scope_name == "Country":
            common_ctry = set(df_d["Country"].unique()) & set(df_a["Country"].unique())
            for c in common_ctry:
                iter_items.append((c, df_d[df_d["Country"]==c], df_a[df_a["Country"]==c]))
        else:
            iter_items.append((scope_name, df_d, df_a))
            
        # PROCESS EACH UNIT
        for c_lbl, sub_d, sub_a in iter_items:
            
            
            # 1. Identify days that have BOTH legs (Long & Short) in Daily data
            # We group by Date and count the 'hedge' rows. Must be 2 (0.0 and 1.0).
            counts_d = sub_d.groupby("DayDate")["hedge"].count()
            valid_dates_d = counts_d[counts_d == 2].index
            
            # 2. Identify days that have BOTH legs in Annual data
            counts_a = sub_a.groupby("DayDate")["hedge"].count()
            valid_dates_a = counts_a[counts_a == 2].index
            
            # 3. Find the Intersection of these "Complete" days
            common_dates = set(valid_dates_d) & set(valid_dates_a)
            
            # 4. Filter the dataframes
            sub_d = sub_d[sub_d["DayDate"].isin(common_dates)].copy()
            sub_a = sub_a[sub_a["DayDate"].isin(common_dates)].copy()
            
            if sub_d.empty or sub_a.empty: continue

            # Cache for Average Anomaly calculation later
            if c_lbl not in avg_cache_daily: 
                avg_cache_daily[c_lbl] = []
                avg_cache_annual[c_lbl] = []
            
            keep = ["DayDate", "hedge", "eret"] + INTERNAL_FACTORS
            avg_cache_daily[c_lbl].append(sub_d[keep].copy())
            avg_cache_annual[c_lbl].append(sub_a[keep].copy())

            res_row = {"Anomaly": anom, "Scope": scope_name, "Country": c_lbl}
            
            base_formula = "eret ~ hedge * (" + " + ".join(INTERNAL_FACTORS) + ")"
            diff_formula = "eret ~ hedge * is_daily * (" + " + ".join(INTERNAL_FACTORS) + ")"
            
            # (I) INFO / DAILY
            m = run_clustered_regression(sub_d, base_formula)
            if m:
                a, se, st = annualized_stats(m.params, m.bse, "hedge")
                res_row.update({"Info_Ret": a, "Info_SE": se, "Info_Star": st})
                
            # (II) FF92 / ANNUAL
            m = run_clustered_regression(sub_a, base_formula)
            if m:
                a, se, st = annualized_stats(m.params, m.bse, "hedge")
                res_row.update({"FF92_Ret": a, "FF92_SE": se, "FF92_Star": st})
                
            # (III) DIFFERENCE (Stacked)
            sub_d_tagged = sub_d.copy(); sub_d_tagged["is_daily"] = 1
            sub_a_tagged = sub_a.copy(); sub_a_tagged["is_daily"] = 0
            stacked = pd.concat([sub_d_tagged, sub_a_tagged], ignore_index=True)
            
            m = run_clustered_regression(stacked, diff_formula)
            if m:
                a, se, st = annualized_stats(m.params, m.bse, "hedge:is_daily")
                res_row.update({"Diff_Ret": a, "Diff_SE": se, "Diff_Star": st})
            
            final_results.append(res_row)
        
        del df_d, df_a, iter_items
        gc.collect()

    # --- AVERAGE ANOMALY ---
    print(f"\n>>> Calculating AVERAGE ANOMALY for {scope_name} ...")
    
    for c_lbl in avg_cache_daily.keys():
        list_d = avg_cache_daily[c_lbl]
        list_a = avg_cache_annual[c_lbl]
        
        if not list_d: continue
        
        res_row = {"Anomaly": "AVERAGE", "Scope": scope_name, "Country": c_lbl}
        
        # Aggregate across all anomalies first
        avg_d = pd.concat(list_d).groupby(["DayDate", "hedge"])[["eret"] + INTERNAL_FACTORS].mean().reset_index()
        avg_a = pd.concat(list_a).groupby(["DayDate", "hedge"])[["eret"] + INTERNAL_FACTORS].mean().reset_index()
        
        
        # 1. Ensure Average Daily has both legs
        counts_d = avg_d.groupby("DayDate")["hedge"].count()
        valid_dates_d = counts_d[counts_d == 2].index
        
        # 2. Ensure Average Annual has both legs
        counts_a = avg_a.groupby("DayDate")["hedge"].count()
        valid_dates_a = counts_a[counts_a == 2].index
        
        # 3. Intersection
        common_avg = set(valid_dates_d) & set(valid_dates_a)
        
        # 4. Filter
        avg_d = avg_d[avg_d["DayDate"].isin(common_avg)].copy()
        avg_a = avg_a[avg_a["DayDate"].isin(common_avg)].copy()
        
        if avg_d.empty or avg_a.empty: continue

        # ... (rest of the average logic: formulas, regressions, etc.)

        base_formula = "eret ~ hedge * (" + " + ".join(INTERNAL_FACTORS) + ")"
        
        # (I) INFO
        m = run_clustered_regression(avg_d, base_formula)
        if m:
            a, se, st = annualized_stats(m.params, m.bse, "hedge")
            res_row.update({"Info_Ret": a, "Info_SE": se, "Info_Star": st})
            
        # (II) FF92
        m = run_clustered_regression(avg_a, base_formula)
        if m:
            a, se, st = annualized_stats(m.params, m.bse, "hedge")
            res_row.update({"FF92_Ret": a, "FF92_SE": se, "FF92_Star": st})
            
        # (III) DIFF
        avg_d["is_daily"] = 1
        avg_a["is_daily"] = 0
        stacked = pd.concat([avg_d, avg_a], ignore_index=True)
        
        diff_formula = "eret ~ hedge * is_daily * (" + " + ".join(INTERNAL_FACTORS) + ")"
        
        m = run_clustered_regression(stacked, diff_formula)
        if m:
            a, se, st = annualized_stats(m.params, m.bse, "hedge:is_daily")
            res_row.update({"Diff_Ret": a, "Diff_SE": se, "Diff_Star": st})
        
        final_results.append(res_row)

    return pd.DataFrame(final_results)

# =================================================================================================
# 5. EXECUTION BLOCK
# =================================================================================================

if __name__ == "__main__":
    
    all_res = []
    
    for sc in ["Total", "Replication", "Country"]:
        try:
            df_res = analyze_scope_safe(sc)
            if not df_res.empty:
                all_res.append(df_res)
        except Exception as e:
            print(f"CRITICAL ERROR in scope {sc}: {e}")

    if not all_res:
        print("No results generated.")
    else:
        master_df = pd.concat(all_res, ignore_index=True)
        
        print(f"\nWriting results to {OUTPUT_EXCEL}...")
        with pd.ExcelWriter(OUTPUT_EXCEL) as writer:
            
            def write_sheet(df_sub, sheet_name):
                if df_sub.empty: return
                out = df_sub.copy()
                
                for prefix in ["FF92", "Info", "Diff"]:
                    if f"{prefix}_Ret" in out.columns:
                        out[f"{prefix}_Ret"] = out[f"{prefix}_Ret"].fillna(0)
                        out[f"{prefix}_SE"] = out[f"{prefix}_SE"].fillna(0)
                        out[f"{prefix} Return"] = out[f"{prefix}_Ret"].map('{:.2f}'.format) + out[f"{prefix}_Star"].fillna("")
                        out[f"{prefix} StdErr"] = "(" + out[f"{prefix}_SE"].map('{:.2f}'.format) + ")"
                
                cols = ["Anomaly"]
                for prefix in ["FF92", "Info", "Diff"]:
                    cols.extend([f"{prefix} Return", f"{prefix} StdErr"])
                
                avg_row = out[out["Anomaly"] == "AVERAGE"]
                others = out[out["Anomaly"] != "AVERAGE"].sort_values("Anomaly")
                final_view = pd.concat([avg_row, others])
                final_cols = [c for c in cols if c in final_view.columns]
                final_view[final_cols].to_excel(writer, sheet_name=sheet_name, index=False)

            avg_view = master_df[master_df["Anomaly"] == "AVERAGE"].copy()
            if not avg_view.empty:
                avg_view.to_excel(writer, sheet_name="Average_Overview", index=False)
            
            write_sheet(master_df[master_df["Scope"] == "Total"], "Summary_Total")
            write_sheet(master_df[master_df["Scope"] == "Replication"], "Summary_Rep")
            
            country_data = master_df[master_df["Scope"] == "Country"]
            if not country_data.empty:
                for c in sorted(country_data["Country"].unique()):
                    sub = country_data[country_data["Country"] == c]
                    write_sheet(sub, f"Summary_C{c}"[:31])
        
        print("\nSUCCESS. Model 1 (Clustered Errors) Complete.")


STARTING SCOPE: Total
[1/27] Processing Anomaly: ACC ...
[2/27] Processing Anomaly: AG ...
[3/27] Processing Anomaly: AT ...
[4/27] Processing Anomaly: CAT ...
[5/27] Processing Anomaly: CPM ...
[6/27] Processing Anomaly: EC ...
[7/27] Processing Anomaly: ES ...
[8/27] Processing Anomaly: GP ...
[9/27] Processing Anomaly: IG ...
[10/27] Processing Anomaly: INV ...
[11/27] Processing Anomaly: LTG ...
[12/27] Processing Anomaly: NCA ...
[13/27] Processing Anomaly: NOA ...
[14/27] Processing Anomaly: NWC ...
[15/27] Processing Anomaly: OL ...
[16/27] Processing Anomaly: OSC ...
[17/27] Processing Anomaly: PM ...
[18/27] Processing Anomaly: POA ...
[19/27] Processing Anomaly: PRO ...
[20/27] Processing Anomaly: PTA ...
[21/27] Processing Anomaly: ROE ...
[22/27] Processing Anomaly: RS ...
[23/27] Processing Anomaly: SG ...
[24/27] Processing Anomaly: SLI ...
[25/27] Processing Anomaly: SLX ...
[26/27] Processing Anomaly: TX ...
[27/27] Processing Anomaly: TXF ...

>>> Calculating AVERAGE 

### Spread Model

In [6]:
import os
import sys
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from pathlib import Path
import gc

# =================================================================================================
# MASTER SCRIPT: CALENDAR-TIME COMPARISON (MODEL 2: SPREAD APPROACH + CONSISTENT SAMPLE)
# =================================================================================================
#
# 1. EXECUTIVE SUMMARY
# --------------------
# This script performs the "Calendar-Time Portfolio" regression analysis using the "Spread" methodology.
# Unlike Model 1 (which stacks Long/Short legs), this model calculates the performance of the 
# Long-Short strategy *before* the regression.
#
#    a) "Info" (Daily Rebalancing): Returns of (Daily Long - Daily Short).
#    b) "FF92" (Annual Rebalancing): Returns of (Annual Long - Annual Short).
#    c) "Difference": The time-series difference of (Daily Spread - Annual Spread).
#
# 2. ECONOMETRIC SPECIFICATION (MODEL 2)
# --------------------------------------
#    * DATA STRUCTURE: Single Time Series.
#      - The data is collapsed to one observation per day (the Strategy Spread).
#      - This isolates the volatility of the "Difference" itself, often providing more precise
#        inference for the gap between strategies than the stacked interaction model.
#
#    * CONSISTENT SAMPLE FIX (CRITICAL):
#      - The script enforces an INNER JOIN between the Daily and Annual datasets.
#      - Regressions for Info, FF92, and Difference are run on the EXACT SAME days.
#      - This ensures logical consistency: Alpha(Info) - Alpha(FF92) ≈ Alpha(Diff).
#      - Without this, the "Info" column might include days where "FF92" is missing, skewing the comparison.
#
#    * REGRESSION FORMULA:
#      Spread ~ 1 + Factors
#      - The Intercept (Alpha) captures the abnormal return of the strategy (or the difference).
#
#    * INFERENCE (NEWEY-WEST HAC):
#      - Standard Errors are estimated using Newey-West (HAC) with 6 Lags.
#      - RATIONALE: Since the data is collapsed to a single time series (one row per date),
#        "Clustering by Date" is mathematically redundant (only 1 item per cluster).
#        The primary risk to inference is Autocorrelation (correlation across time).
#        Newey-West explicitly corrects for this serial correlation.
#
# 3. ANALYSIS SCOPE
# -----------------
#    * Universes: Total (Global), Replication (USA/Can), and Country-Specific.
#    * Factors: Fama-French 5-Factor + Momentum (HML excluded per user config).
#
# 4. SYSTEM ARCHITECTURE
# ----------------------
#    * Memory-safe "Load -> Aggregate -> Discard" cycle.
#    * Strict pathing using 'Temp_file_path_R'.
#    * Single-threaded execution for server stability.
#
# 5. OUTPUT
# ---------
#    * File: CalendarTime_Model2_Spread_Consistent.xlsx
#    * Metrics: Annualized Alpha, Newey-West SE, T-Stats for Info, FF92, and Difference.
# =================================================================================================

# --- 1. CONFIGURATION ---

try:
    DATA_FOLDER = Path(Temp_file_path_R)
except NameError:
    print("Error: 'Temp_file_path_R' variable is not defined.")
    sys.exit("Stopping execution because fallback path is disabled.")

OUTPUT_EXCEL = DATA_FOLDER / "CalendarTime_Model2_Spread_Consistent.xlsx"

os.environ["NUMEXPR_MAX_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

# ANOMALY LIST
ANOMALY_CONFIG = {
    "acc": (1, 10), "ag": (1, 10), "at": (10, 1), "cat": (10, 1),
    "cpm": (10, 1), "ec": (10, 1), "es": (10, 1), "gp": (10, 1),
    "ig": (1, 10), "inv": (1, 10), "ltg": (10, 1), "nca": (1, 10),
    "noa": (1, 10), "nwc": (1, 10), "ol": (10, 1), "osc": (1, 10),
    "pm": (10, 1), "poa": (1, 10), "pro": (10, 1), "pta": (1, 10),
    "roe": (10, 1), "rs": (10, 1), "sg": (1, 10), "sli": (10, 1),
    "slx": (10, 1), "tx": (10, 1), "txf": (1, 10),
}

# PATTERNS
PATTERNS = {
    "Total":       "{stem}_Global_{freq}_clean.parquet",
    "Replication": "{stem}_Replication_{freq}_clean.parquet",
    "Country":     "{stem}_Country_{freq}_clean.parquet"
}

# SETTINGS
INTERNAL_FACTORS = ["mktrf", "smb", "rmw", "cma", "umd"] 
TRADING_DAYS = 252
MIN_COUNTRY_IDS = 30
NW_LAGS = 6

# =================================================================================================
# 2. HELPER FUNCTIONS
# =================================================================================================

def annualized_stats(params, bse, term_name="Intercept"):
    """Calculates Annualized Alpha and SE."""
    if term_name not in params: return np.nan, np.nan, ""
    
    coef = params[term_name]
    se = bse[term_name]
    
    if abs(coef) > 0.5: return np.nan, np.nan, " (Outlier)"

    try:
        ann_coef = (1 + coef)**TRADING_DAYS - 1
        derivative = TRADING_DAYS * (1 + coef)**(TRADING_DAYS - 1)
        ann_se = se * derivative
        
        tstat = coef / se if se != 0 else 0
        stars = ""
        if abs(tstat) >= 2.58: stars = "***"
        elif abs(tstat) >= 1.96: stars = "**"
        elif abs(tstat) >= 1.65: stars = "*"
        
        return ann_coef * 100, ann_se * 100, stars
        
    except (OverflowError, ValueError, RuntimeWarning):
        return np.nan, np.nan, " (Error)"

def make_spread_portfolio(df):
    """
    Converts Stacked DF (Long/Short rows) into Spread DF (Long - Short).
    Returns a DF with ['spread'] and factor columns.
    """
    if df.empty: return None
    # Pivot to separate Long (1.0) and Short (0.0)
    pivoted = df.pivot_table(index="DayDate", columns="hedge", values="eret", aggfunc="mean")
    
    if 1.0 not in pivoted.columns or 0.0 not in pivoted.columns:
        return None
        
    spread_series = pivoted[1.0] - pivoted[0.0]
    
    # Get factors (constant per date)
    factors = df.groupby("DayDate")[INTERNAL_FACTORS].first()
    
    res = pd.DataFrame(spread_series, columns=["spread"])
    res = res.join(factors, how="inner")
    return res

def run_spread_regression(df):
    """
    Regress Spread ~ 1 + Factors.
    Uses Newey-West HAC (Lag 6) because data is a single time series.
    """
    if df.empty or len(df) < 20: return None
    formula = "spread ~ " + " + ".join(INTERNAL_FACTORS)
    try:
        mod = smf.ols(formula, data=df)
        res = mod.fit(cov_type='HAC', cov_kwds={'maxlags': NW_LAGS})
        return res
    except: return None

# =================================================================================================
# 3. DATA LOADER
# =================================================================================================

def get_portfolio_data_optimized(stem, scope, freq, long_dec, short_dec):
    """Loads ONE file, aggregates to portfolio level."""
    fname = PATTERNS[scope].format(stem=stem, freq=freq)
    fpath = DATA_FOLDER / fname
    
    if not fpath.exists(): 
        print(f"      [Warning] File not found: {fname}")
        return None
    
    col_map = {"rf": "RF", "mktrf": "Mkt-RF", "smb": "SMB", "rmw": "RMW", "cma": "CMA", "umd": "Mom"}
    rename_map = {v: k for k, v in col_map.items()}
    file_cols_needed = ["DayDate", "Decile", "ret", col_map["rf"]] + [col_map[f] for f in INTERNAL_FACTORS]
    if scope == "Country": file_cols_needed += ["Country", "ID"]
    
    try:
        df = pd.read_parquet(fpath, columns=file_cols_needed)
        df.rename(columns=rename_map, inplace=True)
        
        df["hedge"] = np.nan
        df.loc[df["Decile"] == long_dec, "hedge"] = 1
        df.loc[df["Decile"] == short_dec, "hedge"] = 0
        df.dropna(subset=["hedge"], inplace=True)
        df["eret"] = df["ret"] - df["rf"]
        
        if scope == "Country":
            df["id_count"] = df.groupby("Country")["ID"].transform("nunique")
            df = df[df["id_count"] >= MIN_COUNTRY_IDS].drop(columns=["id_count", "ID"])
            
        grp_cols = ["DayDate", "hedge"]
        if "Country" in df.columns: grp_cols.append("Country")
        
        port_df = df.groupby(grp_cols)[["eret"] + INTERNAL_FACTORS].mean().reset_index()
        return port_df
        
    except Exception as e:
        print(f"      [Error] Failed reading {fname}: {e}")
        return None

# =================================================================================================
# 4. MAIN LOGIC (CONSISTENT SAMPLE + SPREAD + NEWEY WEST)
# =================================================================================================

def analyze_scope_safe(scope_name):
    print(f"\n" + "="*60)
    print(f"STARTING SCOPE: {scope_name}")
    print("="*60)
    
    final_results = []
    
    # Storage for "Average Anomaly" calculation
    # We store dictionaries containing the 3 aligned DataFrames: {info, ff92, diff}
    avg_cache_diff = [] 
    
    for i, (anom, (l_dec, s_dec)) in enumerate(ANOMALY_CONFIG.items(), 1):
        print(f"[{i}/{len(ANOMALY_CONFIG)}] Processing Anomaly: {anom.upper()} ...")
        
        # A. LOAD
        df_d = get_portfolio_data_optimized(anom, scope_name, "Daily", l_dec, s_dec)
        df_a = get_portfolio_data_optimized(anom, scope_name, "Annual", l_dec, s_dec)
        gc.collect() 
        
        if df_d is None or df_a is None: continue

        # B. DEFINE ITERATION UNITS
        iter_items = []
        if scope_name == "Country":
            common_ctry = set(df_d["Country"].unique()) & set(df_a["Country"].unique())
            for c in common_ctry:
                iter_items.append((c, df_d[df_d["Country"]==c], df_a[df_a["Country"]==c]))
        else:
            iter_items.append((scope_name, df_d, df_a))
            
        # C. PROCESS EACH UNIT
        for c_lbl, sub_d, sub_a in iter_items:
            
            # 1. MAKE SPREADS (Long - Short)
            spr_d = make_spread_portfolio(sub_d)
            spr_a = make_spread_portfolio(sub_a)
            if spr_d is None or spr_a is None: continue

            # --- START CHANGE: KEEP BOTH FACTOR SETS ---

            # 2. CONSISTENT SAMPLE FIX (INNER JOIN)
            # Use suffixes to keep BOTH Daily (_d) and Annual (_a) factors
            combined = spr_d.join(spr_a, lsuffix="_d", rsuffix="_a", how="inner")
            
            if combined.empty: continue
            
            # Define dictionaries to rename columns back to standard names (mktrf, smb, etc.)
            rename_daily  = {f"{f}_d": f for f in INTERNAL_FACTORS}
            rename_annual = {f"{f}_a": f for f in INTERNAL_FACTORS}
            
            # 3. CONSTRUCT THE 3 REGRESSION DATASETS
            
            # (I) INFO: Daily Spread ~ Daily Factors
            df_info = combined.copy()
            df_info.rename(columns=rename_daily, inplace=True)  # Activate Daily Factors
            df_info["spread"] = df_info["spread_d"]
            
            # (II) FF92: Annual Spread ~ ANNUAL Factors
            df_ff92 = combined.copy()
            df_ff92.rename(columns=rename_annual, inplace=True) # Activate Annual Factors
            df_ff92["spread"] = df_ff92["spread_a"]
            
            # (III) DIFF: (Daily - Annual) ~ DAILY Factors
            # We use Daily Factors as the benchmark for the difference
            df_diff = combined.copy()
            df_diff.rename(columns=rename_daily, inplace=True)  # Activate Daily Factors
            df_diff["spread"] = df_diff["spread_d"] - df_diff["spread_a"]
            
            # 4. CACHE FOR AVERAGE (Store these aligned datasets)
            avg_cache_diff.append({
                "country": c_lbl, 
                "info": df_info[["spread"] + INTERNAL_FACTORS],
                "ff92": df_ff92[["spread"] + INTERNAL_FACTORS],
                "diff": df_diff[["spread"] + INTERNAL_FACTORS]
            })

            # 5. RUN REGRESSIONS (NEWEY WEST)
            res_row = {"Anomaly": anom, "Scope": scope_name, "Country": c_lbl}
            
            # Info Regression
            m = run_spread_regression(df_info)
            if m:
                a, se, st = annualized_stats(m.params, m.bse)
                res_row.update({"Info_Ret": a, "Info_SE": se, "Info_Star": st})
                
            # FF92 Regression
            m = run_spread_regression(df_ff92)
            if m:
                a, se, st = annualized_stats(m.params, m.bse)
                res_row.update({"FF92_Ret": a, "FF92_SE": se, "FF92_Star": st})
                
            # Diff Regression
            m = run_spread_regression(df_diff)
            if m:
                a, se, st = annualized_stats(m.params, m.bse)
                res_row.update({"Diff_Ret": a, "Diff_SE": se, "Diff_Star": st})
            
            final_results.append(res_row)
        
        del df_d, df_a, iter_items
        gc.collect()

    # --- AVERAGE ANOMALY (CONSISTENT) ---
    print(f"\n>>> Calculating AVERAGE ANOMALY for {scope_name} ...")
    
    # Re-organize cache by Country
    country_caches = {}
    for item in avg_cache_diff:
        c = item["country"]
        if c not in country_caches: country_caches[c] = {"info": [], "ff92": [], "diff": []}
        country_caches[c]["info"].append(item["info"])
        country_caches[c]["ff92"].append(item["ff92"])
        country_caches[c]["diff"].append(item["diff"])
        
    for c_lbl, dfs in country_caches.items():
        if not dfs["info"]: continue
        
        # Calculate Average Spread across anomalies (on aligned dates)
        # We concat and groupby index (Date) to get the mean
        avg_info = pd.concat(dfs["info"]).groupby(level=0).mean()
        avg_ff92 = pd.concat(dfs["ff92"]).groupby(level=0).mean()
        avg_diff = pd.concat(dfs["diff"]).groupby(level=0).mean()
        
        res_row = {"Anomaly": "AVERAGE", "Scope": scope_name, "Country": c_lbl}
        
        # (I) INFO
        m = run_spread_regression(avg_info)
        if m:
            a, se, st = annualized_stats(m.params, m.bse)
            res_row.update({"Info_Ret": a, "Info_SE": se, "Info_Star": st})
            
        # (II) FF92
        m = run_spread_regression(avg_ff92)
        if m:
            a, se, st = annualized_stats(m.params, m.bse)
            res_row.update({"FF92_Ret": a, "FF92_SE": se, "FF92_Star": st})
            
        # (III) DIFF
        m = run_spread_regression(avg_diff)
        if m:
            a, se, st = annualized_stats(m.params, m.bse)
            res_row.update({"Diff_Ret": a, "Diff_SE": se, "Diff_Star": st})
            
        final_results.append(res_row)

    return pd.DataFrame(final_results)

# =================================================================================================
# 5. EXECUTION BLOCK
# =================================================================================================

if __name__ == "__main__":
    
    all_res = []
    
    for sc in ["Total", "Replication", "Country"]:
        try:
            df_res = analyze_scope_safe(sc)
            if not df_res.empty:
                all_res.append(df_res)
        except Exception as e:
            print(f"CRITICAL ERROR in scope {sc}: {e}")

    if not all_res:
        print("No results generated.")
    else:
        master_df = pd.concat(all_res, ignore_index=True)
        
        print(f"\nWriting results to {OUTPUT_EXCEL}...")
        with pd.ExcelWriter(OUTPUT_EXCEL) as writer:
            
            def write_sheet(df_sub, sheet_name):
                if df_sub.empty: return
                out = df_sub.copy()
                
                for prefix in ["FF92", "Info", "Diff"]:
                    if f"{prefix}_Ret" in out.columns:
                        out[f"{prefix}_Ret"] = out[f"{prefix}_Ret"].fillna(0)
                        out[f"{prefix}_SE"] = out[f"{prefix}_SE"].fillna(0)
                        out[f"{prefix} Return"] = out[f"{prefix}_Ret"].map('{:.2f}'.format) + out[f"{prefix}_Star"].fillna("")
                        out[f"{prefix} StdErr"] = "(" + out[f"{prefix}_SE"].map('{:.2f}'.format) + ")"
                
                cols = ["Anomaly"]
                for prefix in ["FF92", "Info", "Diff"]:
                    cols.extend([f"{prefix} Return", f"{prefix} StdErr"])
                
                avg_row = out[out["Anomaly"] == "AVERAGE"]
                others = out[out["Anomaly"] != "AVERAGE"].sort_values("Anomaly")
                final_view = pd.concat([avg_row, others])
                final_cols = [c for c in cols if c in final_view.columns]
                final_view[final_cols].to_excel(writer, sheet_name=sheet_name, index=False)

            avg_view = master_df[master_df["Anomaly"] == "AVERAGE"].copy()
            if not avg_view.empty:
                avg_view.to_excel(writer, sheet_name="Average_Overview", index=False)
            
            write_sheet(master_df[master_df["Scope"] == "Total"], "Summary_Total")
            write_sheet(master_df[master_df["Scope"] == "Replication"], "Summary_Rep")
            
            country_data = master_df[master_df["Scope"] == "Country"]
            if not country_data.empty:
                for c in sorted(country_data["Country"].unique()):
                    sub = country_data[country_data["Country"] == c]
                    write_sheet(sub, f"Summary_C{c}"[:31])
        
        print("\nSUCCESS. Model 2 (Spread + Consistent Sample) Complete.")


STARTING SCOPE: Total
[1/27] Processing Anomaly: ACC ...
[2/27] Processing Anomaly: AG ...
[3/27] Processing Anomaly: AT ...
[4/27] Processing Anomaly: CAT ...
[5/27] Processing Anomaly: CPM ...
[6/27] Processing Anomaly: EC ...
[7/27] Processing Anomaly: ES ...
[8/27] Processing Anomaly: GP ...
[9/27] Processing Anomaly: IG ...
[10/27] Processing Anomaly: INV ...
[11/27] Processing Anomaly: LTG ...
[12/27] Processing Anomaly: NCA ...
[13/27] Processing Anomaly: NOA ...
[14/27] Processing Anomaly: NWC ...
[15/27] Processing Anomaly: OL ...
[16/27] Processing Anomaly: OSC ...
[17/27] Processing Anomaly: PM ...
[18/27] Processing Anomaly: POA ...
[19/27] Processing Anomaly: PRO ...
[20/27] Processing Anomaly: PTA ...
[21/27] Processing Anomaly: ROE ...
[22/27] Processing Anomaly: RS ...
[23/27] Processing Anomaly: SG ...
[24/27] Processing Anomaly: SLI ...
[25/27] Processing Anomaly: SLX ...
[26/27] Processing Anomaly: TX ...
[27/27] Processing Anomaly: TXF ...

>>> Calculating AVERAGE 